In [14]:
import datetime, itertools, sys
import pandas as ps
from subprocess import Popen, PIPE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.cluster import KMeans
from scipy.optimize import minimize
from pymystem3 import Mystem
import nltk

In [2]:
data = ps.read_csv("data/spelled-f.csv", sep=';', header=None,
                   index_col=0,names=['id','title','text','cluster','date','publisher'])

## Предварительная очистка данных

In [3]:
data = data[~data["cluster"].isin(["-", "S", "Standard "])]
print("Число записей в таблице:", len(data))

Число записей в таблице: 32317


In [4]:
day_counts = {}
for _, row in data.iterrows():
    day_counts[row["date"]] = day_counts.get(row["date"], 0) + 1
max_date = max((count, date) for date, count in day_counts.items())[1]
print("Самый плотный день - {}, число записей: {}".format(max_date, day_counts[max_date]))

Самый плотный день - 2016-01-27, число записей: 19846


In [9]:
appropriate_days = [date for date, count in day_counts.items() if 20 <= count <= 30]
ch_day = appropriate_days[0]
data_1d = data[data["date"] == ch_day]
print("Выбранный день - {}, число записей: {}".format(ch_day, day_counts[ch_day]))

Выбранный день - 2015-09-26, число записей: 25


## Лемматизация текстов

In [10]:
m = Mystem()

def do_stem(df):
    cluster_index = {cluster: i for i, cluster in enumerate(df["cluster"].unique())}
    messages = ["".join(m.lemmatize(row["title"] + ". " + row["text"])) for _, row in df.iterrows()]
    clusters = [cluster_index[row["cluster"]] for _, row in df.iterrows()]
    return messages, clusters, cluster_index

In [11]:
messages, clusters, cluster_index = do_stem(data_1d)
print(messages[0])
print(len(messages), len(clusters))

польский мид вызывать для разговор посол россия после его замечание о российский-польский отношение. поляк с удивление и тревога воспринимать высказывать взгляд посол россия в варшава сергей андреев на причина начало второй мировой война.польский министерство иностранный дело вызывать для...

25 25


## Построение матрицы TF*IDF

In [12]:
tfidf_vectorizer = TfidfVectorizer(stop_words=nltk.corpus.stopwords.words('russian'))
tfidf_matrix = tfidf_vectorizer.fit_transform(messages)
print(tfidf_matrix.shape)

(25, 390)


## Эксперимент с кластеризацией

In [13]:
print("Количество кластеров:", len(set(clusters)))

Количество кластеров: 11


In [22]:
def marks_to_pairwise(y_cls, p_cls):
    assert len(y_cls) == len(p_cls)
    res = {"ids": [], "y": [], "p": []}
    for i1, i2 in itertools.combinations(range(len(y_cls)), 2):
        res["ids"].append(sorted((i1, i2)))
        res["y"].append(bool(y_cls[i1] == y_cls[i2]))
        res["p"].append(bool(p_cls[i1] == p_cls[i2]))
    return ps.DataFrame(res, index=None)

In [17]:
def cross_class_report(res):
    classes = res["y"].unique()
    table = ps.DataFrame(index=classes, columns=classes)
    for true_cls in classes:
        tmp = res[res["y"] == true_cls]
        for pred_cls in classes:
            table[pred_cls][true_cls] = len(tmp[tmp["p"] == pred_cls])
    return table

In [36]:
def print_performance(tfidf_matrix, clusters, n_clusters, seed=42):
    km = KMeans(n_clusters=n_clusters, random_state=seed)
    km.fit(tfidf_matrix)
    res = marks_to_pairwise(clusters, km.labels_.tolist())
    print(classification_report(res["y"], res["p"]))
    print(cross_class_report(res), "\n\n")
    return res

In [26]:
print_performance(tfidf_matrix, clusters, 11)

             precision    recall  f1-score   support

      False       1.00      1.00      1.00       283
       True       1.00      1.00      1.00        17

avg / total       1.00      1.00      1.00       300

      False True 
False   283     0
True      0    17 




## Кластеризация для нескольких дней

In [27]:
def get_prec_recall_f(res, cls):
    pred = res[res["p"] == cls]
    prec = len(pred[pred["y"] == cls]) / len(pred)
    act = res[res["y"] == cls]
    rec = len(act[act["p"] == cls]) / len(act)
    return prec, rec, 2/(1/prec + 1/rec)

In [28]:
def get_data_for_day(day):
    data_1d = data[data["date"] == day]
    messages, clusters, cluster_index = do_stem(data_1d)
    
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(messages)
    
    return tfidf_matrix, clusters

In [47]:
days_to_check = ['2015-11-29', '2015-09-26', '2015-09-28', '2015-03-28', '2015-08-30',
                '2015-10-25', '2015-06-25', '2015-05-25', '2015-05-26', '2015-05-30']
scores = []
for day in days_to_check:
    tfidf_matrix, clusters = get_data_for_day(day)
    n_clusters = len(set(clusters))
    print("День: {}\nЧисло новостей: {}\nКластеров:{}".format(day, tfidf_matrix.shape[0], n_clusters))
    res = print_performance(tfidf_matrix, clusters, n_clusters, 42) 
    prec, recall, f = get_prec_recall_f(res, 1)
    scores.append((prec, recall, f, day_counts[day]))

День: 2015-11-29
Число новостей: 15
Кластеров:8
             precision    recall  f1-score   support

      False       0.97      0.97      0.97        96
       True       0.67      0.67      0.67         9

avg / total       0.94      0.94      0.94       105

      False True 
False    93     3
True      3     6 


День: 2015-09-26
Число новостей: 25
Кластеров:11
             precision    recall  f1-score   support

      False       1.00      1.00      1.00       283
       True       1.00      1.00      1.00        17

avg / total       1.00      1.00      1.00       300

      False True 
False   283     0
True      0    17 


День: 2015-09-28
Число новостей: 51
Кластеров:21
             precision    recall  f1-score   support

      False       1.00      1.00      1.00      1236
       True       0.86      0.92      0.89        39

avg / total       0.99      0.99      0.99      1275

      False True 
False  1230     6
True      3    36 


День: 2015-03-28
Число новостей: 75
Кл

In [48]:
total_pairs = sum(s[3]*(s[3]-1)//2 for s in scores)
avg_prec = sum(prec*(count*(count-1)//2)/total_pairs for prec, _, _, count in scores)
avg_rec = sum(rec*(count*(count-1)//2)/total_pairs for _, rec, _, count in scores)
avg_f = 2/(1/avg_prec + 1/avg_rec)
print("Средняя точность:", avg_prec)
print("Средняя полнота:", avg_rec)
print("Средняя Ф-мера:", avg_f)

Средняя точность: 0.6355091418602679
Средняя полнота: 0.7851515888484846
Средняя Ф-мера: 0.7024492219340714
